# 项目代码结构分析 - 2D 流体预测 (FNO & UNet)

## 📋 项目概述

本项目使用 **CFDBench** 数据集训练神经网络来预测二维流体动力学演化。核心任务是学习离散的 Navier-Stokes 流图：

$$u^{t+\Delta t}(x) = G_\theta(u^t(x), BC, \nu, \rho, geometry)$$

其中：
- $u$: 速度场 (u, v 分量)
- $BC$: 边界条件
- $\nu$: 粘度
- $\rho$: 密度
- $geometry$: 几何形状 (圆柱、空腔、大坝、管道)

---

## 📁 目录结构

```
physics188Capstone/
├── data/                          # 数据目录
│   └── CFDBench/                  # CFDBench 数据集
│       ├── cylinder/              # 圆柱绕流
│       ├── cavity/                # 空腔流动
│       ├── dam/                   # 大坝溃坝
│       └── tube/                  # 管道流动
│
├── src/                           # 源代码
│   ├── models/                    # 模型定义
│   │   ├── FNO.py                 # Fourier Neural Operator
│   │   └── UNet_lagecy.py         # UNet 模型
│   │
│   ├── data_handle/               # 数据处理
│   │   ├── data.py                # 简单数据加载器
│   │   └── testdata.py            # 高级数据集类 (CFDBenchDataset)
│   │
│   ├── training/                  # 训练相关
│   │   ├── trainer_general.py     # 通用训练器
│   │   ├── FNO.ipynb              # FNO 训练笔记本
│   │   ├── UNet.ipynb             # UNet 训练笔记本
│   │   ├── data.py                # 数据加载函数
│   │   ├── UNet_DOCUMENTATION.md  # UNet 详细文档
│   │   └── epochs/                # 模型权重保存
│   │       ├── FNO_large/         # FNO 大模型权重
│   │       └── ...
│   │
│   ├── Visualize/                 # 可视化
│   │   └── Visualize.ipynb        # 可视化笔记本 (当前文件)
│   │
│   └── train.ipynb                # 主训练入口
│
└── ToDos/                         # 待办事项
    └── VisualizationToDo.md
```

## 🧠 模型架构

### 1. FNO (Fourier Neural Operator) - `src/models/FNO.py`

**核心思想**: 在频域中进行卷积操作，学习全局特征

```
输入 [B, H, W, 10] → Lifting → FNO Blocks × N → Projection → 输出 [B, H, W, 2]
                    (Linear)   (Spectral Conv)    (Linear)
```

**输入通道 (10维)**:
- `u, v`: 速度分量 (2)
- `sdf`: 有符号距离函数 (1)
- `log_rho, log_mu`: 对数密度和粘度 (2)
- `vel_bc`: 边界速度 (1)
- `geom_onehot`: 几何类型编码 (4)

**输出通道 (2维)**: `u, v` 预测速度

**关键组件**:
| 组件 | 描述 |
|------|------|
| `SpectralConv2d` | FFT → 频域乘法 → IFFT |
| `FNOBlock` | 谱卷积 + 逐点卷积 + 残差连接 |
| `Lifting` | 输入投影到高维空间 |
| `Projection` | 从高维投影回输出空间 |

**超参数**:
- `width=32`: 隐藏层宽度
- `modes1=12, modes2=12`: 保留的傅里叶模态数
- `n_layers=4`: FNO 块数量
- `padding=8`: 边界填充

---

### 2. UNet - `src/models/UNet_lagecy.py`

**核心思想**: 编码器-解码器结构 + 跳跃连接

```
输入 → [编码器] → [瓶颈层] → [解码器] → 输出
          ↓_________skip connections_________↑
```

**架构**:
- **编码器**: DoubleConv → MaxPool (4层: 64→128→256→512)
- **瓶颈层**: 512 → 1024
- **解码器**: ConvTranspose2d → DoubleConv (4层: 1024→64)

**优势**: 在 Mac 系统上更快 (不需要 FFT 操作)

## 📊 数据处理

### 数据集类 - `src/data_handle/testdata.py`

**`CFDBenchDataset`** - 主要数据集类

**支持的几何类型**:
| 几何类型 | 描述 | SDF 计算 |
|----------|------|----------|
| `cylinder` | 圆柱绕流 | 到圆柱表面的距离 |
| `cavity` | 盖驱动空腔 | 到最近墙壁的距离 |
| `dam` | 大坝溃坝 | 障碍物区域标记 |
| `tube` | 管道流动 | 到上下壁面的距离 |

**数据格式**:
- 每个案例包含: `u.npy`, `v.npy`, `case.json`
- 速度场形状: `[T, 64, 64]` (T=时间步数)
- 空间分辨率: 64×64 网格

**输入特征构建** (`__getitem__`):
```python
x = [u, v, sdf, log(rho), log(mu), vel_bc, geom_onehot×4]  # shape: [64, 64, 10]
y = [u_next, v_next]  # shape: [rollout_steps, 64, 64, 2]
```

**关键功能**:
- `preload_all()`: 预加载所有数据到 RAM
- `stratified_split()`: 按案例分层划分 (防止时序泄漏)
- `denormalize()`: 反归一化预测结果

---

### 简单数据加载器 - `src/data_handle/data.py`

**`Data`** - 简化版数据集类

提取的元数据参数:
- `vel_in`: 入口速度
- `density`, `viscosity`: 物理属性
- `radius`, `x_min`, `x_max`, `y_min`, `y_max`: 几何参数

## 🏋️ 训练系统

### 通用训练器 - `src/training/trainer_general.py`

**`Trainer` 类** - 完整的训练流程管理

**初始化参数**:
```python
Trainer(
    model,              # FNO 或 UNet 模型
    dataset,            # CFDBenchDataset
    epochs=50,          # 训练轮数
    batch_size=16,      # 批次大小
    lr=1e-4,            # 学习率
    val_split=0.1,      # 验证集比例
    device='cuda/mps/cpu'  # 计算设备
)
```

**训练模式**:

| 方法 | 描述 |
|------|------|
| `train_single_step()` | 单步预测训练 (预测下一时间步) |
| `train_autoregressive()` | 自回归训练 (多步滚动预测) |

**损失函数**:
1. **能量加权 MSE** (`_energy_weighted_loss`):
   $$L = MSE(pred, target) + \lambda \cdot MSE(KE_{pred}, KE_{target})$$
   其中 $KE = \frac{1}{2}(u^2 + v^2)$

2. **可压缩性惩罚** (`compressible_loss`):
   $$L_{div} = \omega \cdot \mathbb{E}[(\nabla \cdot \mathbf{u})^2]$$
   强制不可压缩流: $\frac{\partial u}{\partial x} + \frac{\partial v}{\partial y} \approx 0$

**评估指标** (`evaluate()`):
- MSE / RMSE: 均方误差
- MAE: 平均绝对误差
- Energy MAE: 动能误差

---

### 训练笔记本

**FNO.ipynb** - FNO 模型训练
- 数据加载 (BC 子集)
- 几何掩码创建
- 单步/自回归训练
- 可视化预测结果

**UNet.ipynb** - UNet 模型训练
- 类似 FNO 的流程
- 包含详细的性能评估

## 💾 已保存的模型权重

**位置**: `src/training/epochs/`

```
epochs/
├── FNO_large/                    # FNO 大模型
│   ├── best_model.pt             # 单步训练最佳模型
│   ├── best_model_ar.pt          # 自回归训练最佳模型
│   └── training_log.txt          # 训练日志
│
├── FNO_1/                        # FNO 实验1
│   ├── FNO_no_rollout/
│   │   └── model_weights_6       # 第6轮权重
│   └── rollout/
│       └── model_weights_0       # 滚动训练权重
│
├── FNO_no_rollout/               # 无滚动训练
│   └── model_weights_6
│
└── rollout/                      # 滚动训练
    └── model_weights_0
```

---

## 🔄 数据流总结

```
┌─────────────────────────────────────────────────────────────────┐
│                        训练流程                                  │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  CFDBench Data                                                  │
│  (u.npy, v.npy, case.json)                                     │
│         ↓                                                       │
│  CFDBenchDataset                                                │
│  - 计算 SDF (有符号距离函数)                                     │
│  - 提取物理参数                                                  │
│  - 构建输入特征 [64, 64, 10]                                    │
│         ↓                                                       │
│  DataLoader (batch_size=16)                                     │
│         ↓                                                       │
│  ┌──────────────┐     ┌──────────────┐                         │
│  │     FNO      │     │    UNet      │                         │
│  │  (频域卷积)   │ OR  │ (编码解码器)  │                         │
│  └──────────────┘     └──────────────┘                         │
│         ↓                                                       │
│  预测下一时间步 [64, 64, 2]                                      │
│         ↓                                                       │
│  损失计算: MSE + Energy + Divergence                            │
│         ↓                                                       │
│  反向传播 + 优化器更新                                           │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

## 📈 待完成的可视化任务

根据 `ToDos/VisualizationToDo.md`:

- [ ] **几何可视化**
  - [ ] FNO: 圆柱
  - [ ] FNO-large: 所有几何形状
  - [ ] UNet: 圆柱的不同边界条件

- [ ] **速度流场可视化**: 所有几何形状
  - [ ] 随时间演化 (时间步)

- [ ] **残差分析**: FNO / FNO-large / UNet

- [ ] **模型结构图**

- [ ] **结果基准对比**

---

## 🔧 使用方法

### 加载模型示例:

```python
import sys
sys.path.append('../')

from models.FNO import FNO
import torch

# 创建模型
model = FNO(
    in_channels=10,
    out_channels=2,
    width=32,
    modes1=12,
    modes2=12,
    n_layers=4
)

# 加载权重
model.load_state_dict(torch.load('../training/epochs/FNO_large/best_model.pt'))
model.eval()
```

### 加载数据示例:

```python
from data_handle.testdata import CFDBenchDataset, stratified_split

dataset = CFDBenchDataset(
    data_root='../../data/CFDBench',
    geometries='all',  # 或 ['cylinder', 'cavity']
    rollout_steps=1,
    normalize=True
)

train_set, val_set = stratified_split(dataset, val_fraction=0.15)
```

## 📚 关键文件快速参考

| 文件 | 用途 | 关键类/函数 |
|------|------|-------------|
| `models/FNO.py` | FNO 模型定义 | `FNO`, `SpectralConv2d`, `FNOBlock` |
| `models/UNet_lagecy.py` | UNet 模型定义 | `UNet`, `DoubleConv` |
| `data_handle/testdata.py` | 高级数据集 | `CFDBenchDataset`, `stratified_split` |
| `data_handle/data.py` | 简单数据集 | `Data` |
| `training/trainer_general.py` | 训练器 | `Trainer` |
| `training/FNO.ipynb` | FNO 训练流程 | - |
| `training/UNet.ipynb` | UNet 训练流程 | - |

---

## 🎯 下一步

以下单元格将开始实际的可视化代码...

## 🔍 FNO 模型使用的几何详情

### 训练数据来源

根据 `FNO.ipynb` 分析，FNO 模型主要使用以下数据：

| 模型版本 | 几何类型 | 数据子集 | 训练配置 |
|----------|----------|----------|----------|
| **FNO (FNO.ipynb)** | Cylinder (圆柱) | `bc` (边界条件) | width=64, modes=16, layers=4 |
| **FNO_1** | Cylinder | `bc` | width=32, modes=16, layers=3 |
| **FNO_large** | **所有几何** (cylinder, cavity, dam, tube) | 全部 | 使用 `CFDBenchDataset` |

---

### Cylinder (圆柱绕流) - 主要几何

这是 FNO.ipynb 中使用的**唯一几何类型**，数据来自 `bc` 子集。

**case.json 参数结构**:
```json
{
    "vel_in": 1.0,       // 入口速度 (m/s)
    "density": 1.0,      // 流体密度 (kg/m³)
    "viscosity": 0.01,   // 动力粘度 (Pa·s)
    "radius": 0.5,       // 圆柱半径 (圆心在原点)
    "x_min": -4.0,       // 计算域 X 最小边界
    "x_max": 12.0,       // 计算域 X 最大边界
    "y_min": -4.0,       // 计算域 Y 最小边界
    "y_max": 4.0         // 计算域 Y 最大边界
}
```

**几何示意图**:
```
        y_max ────────────────────────────
              │                          │
              │    ●────→ 流动方向        │
              │   ╱╲                     │
        0 ────│──●  ● (圆柱, r=radius)   │
              │   ╲╱                     │
              │                          │
        y_min ────────────────────────────
              x_min        0           x_max
                     inlet
```

**输入通道 (11维) - FNO.ipynb 版本**:
| 通道 | 名称 | 描述 |
|------|------|------|
| 0 | `u` | x 方向速度 |
| 1 | `v` | y 方向速度 |
| 2 | `u_B` | 边界速度 (vel_in) |
| 3 | `ρ` | 密度 |
| 4 | `μ` | 粘度 |
| 5 | `d` | 直径 (2×radius) |
| 6 | `x_min` | 域边界 |
| 7 | `x_max` | 域边界 |
| 8 | `y_min` | 域边界 |
| 9 | `y_max` | 域边界 |
| 10 | `mask` | 几何掩码 (0=固体, 1=流体) |

---

### FNO_large 支持的全部几何类型

FNO_large 使用 `CFDBenchDataset` (在 `testdata.py` 中定义)，支持 4 种几何：

#### 1. Cylinder (圆柱绕流)
```python
参数: {x_min, x_max, y_min, y_max, radius, vel_in, density, viscosity}
SDF:  sqrt(x² + y²) - radius  (圆柱中心在原点)
```

#### 2. Cavity (盖驱动空腔)
```python
参数: {vel_top, density, viscosity, rotated}
SDF:  min(x, 1-x, y, 1-y)  (到最近墙壁的距离)
特点: 顶部以 vel_top 速度移动驱动流动
```

#### 3. Dam (大坝溃坝)
```python
参数: {barrier_height, barrier_width, height, width, velocity, density, viscosity}
SDF:  障碍物区域标记 (-0.1 内部, 0.5 外部)
特点: 包含底部障碍物的自由表面流动
```

#### 4. Tube (管道流动)
```python
参数: {vel_in, density, viscosity}
SDF:  min(y, 1-y)  (到上下壁面的距离)
特点: 简单的层流/湍流管道流动
```

---

### 参数差异汇总

| 参数 | Cylinder | Cavity | Dam | Tube |
|------|:--------:|:------:|:---:|:----:|
| `x_min, x_max` | ✅ | ❌ | ❌ | ❌ |
| `y_min, y_max` | ✅ | ❌ | ❌ | ❌ |
| `radius` | ✅ | ❌ | ❌ | ❌ |
| `vel_in` | ✅ | ❌ | ❌ | ✅ |
| `vel_top` | ❌ | ✅ | ❌ | ❌ |
| `velocity` | ❌ | ❌ | ✅ | ❌ |
| `barrier_height/width` | ❌ | ❌ | ✅ | ❌ |
| `height, width` | ❌ | ❌ | ✅ | ❌ |
| `rotated` | ❌ | ⚪ | ❌ | ❌ |
| `density` | ✅ | ✅ | ✅ | ✅ |
| `viscosity` | ✅ | ✅ | ✅ | ✅ |

---

### 训练历史 (FNO_large)

根据 `training_log.txt`：

| 阶段 | 训练类型 | Epochs | 最终 Train Loss | 最终 Val Loss |
|------|----------|--------|-----------------|---------------|
| Part 1 | Single-step | 11 | 0.0077 | 0.0133 |
| Part 2 | Single-step | 22 | 0.0097 | 0.0233 |
| Part 3 | Single-step | 19 | 0.00075 | 0.00077 |
| Part 4 | **Autoregressive (rollout=5)** | 30 | 0.00186 | 0.00157 |

**最佳模型**: `best_model_ar.pt` (自回归训练, Val Loss = 0.00157)